## BDAD Project  - Team D

#### Sentiment Classification

In [1]:
# # |
# # Install pyspark
# ! pip install pyspark==2.4.4

# # Install Spark NLP
# ! pip install --ignore-installed spark-nlp

In [2]:
## import dependencies
import findspark
findspark.init()

import pyspark
pyspark.version

<module 'pyspark.version' from 'C:\\spark\\spark-2.4.5-bin-hadoop2.7\\python\\pyspark\\version.py'>

In [3]:
# !pip install findspark 

In [4]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version", spark.version)


Spark NLP version:  2.5.5
Apache Spark version 2.4.5


downloading IMDB movie reviews dataset for training our multi-class sentiment classifier

In [4]:
!wget -O aclimdb_train.csv https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/sentiment-corpus/aclimdb/aclimdb_train.csv

--2020-08-25 15:34:41--  https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/sentiment-corpus/aclimdb/aclimdb_train.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.88.101
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.88.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33497180 (32M) [text/csv]
Saving to: 'aclimdb_train.csv'

     0K .......... .......... .......... .......... ..........  0%  111K 4m56s
    50K .......... .......... .......... .......... ..........  0% 1.10M 2m42s
   100K .......... .......... .......... .......... ..........  0%  205K 2m41s
   150K .......... .......... .......... .......... ..........  0% 10.7M 2m1s
   200K .......... .......... .......... .......... ..........  0%  210K 2m8s
   250K .......... .......... .......... .......... ..........  0% 7.98M 1m47s
   300K .......... .......... .......... .......... ..........  1% 6.12M 92s
   350K .......... .......... .......... .......... ..

In [5]:
!wget -O aclimdb_test.csv https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/sentiment-corpus/aclimdb/aclimdb_test.csv

--2020-08-25 15:34:49--  https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/sentiment-corpus/aclimdb/aclimdb_test.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.18.115
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.18.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32715164 (31M) [text/csv]
Saving to: 'aclimdb_test.csv'

     0K .......... .......... .......... .......... ..........  0%  102K 5m14s
    50K .......... .......... .......... .......... ..........  0% 9.53M 2m38s
   100K .......... .......... .......... .......... ..........  0%  205K 2m37s
   150K .......... .......... .......... .......... ..........  0% 10.7M 1m58s
   200K .......... .......... .......... .......... ..........  0%  209K 2m5s
   250K .......... .......... .......... .......... ..........  0% 9.64M 1m44s
   300K .......... .......... .......... .......... ..........  1% 11.2M 90s
   350K .......... .......... .......... .......... ...

In [6]:
# !head -n 5 aclimdb_train.csv

'head' is not recognized as an internal or external command,
operable program or batch file.


The content is inside `text` column and the sentiment is inside `label` column

In [10]:
trainDataset = spark.read \
              .option("header",True)\
              .csv("aclimdb_train.csv")

In [11]:
trainDataset.show()

+--------------------+--------+
|                text|   label|
+--------------------+--------+
|This is an Excell...|positive|
|The Sarah Silverm...|positive|
|"Prom Night" is a...|negative|
|So often a band w...|positive|
|"Pet Sematary" is...|positive|
|I watched the fil...|negative|
|Boy this movie ha...|negative|
|Checking the spoi...|negative|
|Despite its rathe...|positive|
|Absolute masterpi...|positive|
|The tweedy profes...|positive|
|A movie best summ...|negative|
|Take young, prett...|negative|
|For months I've b...|negative|
|"Batman: The Myst...|positive|
|Well, it was funn...|negative|
|I have seen the s...|positive|
|Brainless film ab...|negative|
|Leave it to geniu...|negative|
|Seven Pounds star...|positive|
+--------------------+--------+
only showing top 20 rows



In [12]:
type(trainDataset)

pyspark.sql.dataframe.DataFrame

In [13]:
trainDataset.count()

25000

In [14]:
# importing dependencies for creating pipeline
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

In [34]:
# actual content is inside description column

## setting up the trainin pipeline

document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

## sentence encoder 
use = UniversalSentenceEncoder.pretrained() \
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
sentimentdl = SentimentDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("label")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)


## build the pipeline
pipeline = Pipeline(
    stages = [
        document,
        use,
        sentimentdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [35]:
pipelineModel = pipeline.fit(trainDataset)   ## fit pipeline

In [19]:
# !cd ~/annotator_logs && ls -l

In [20]:
# !cat ~/annotator_logs/SentimentDLApproach_2ea7dc3149c2.log

#### (optional) use only when you want to save your pipeline



In [15]:
load_df = spark.createDataFrame([
    "This movie is a delight for those of all ages. I have seen it several times and each time I am enchanted by the characters and magic. The cast is outstanding, the special effects delightful, everything most believable.",
    "This film was to put it simply rubbish. The child actors couldn't act, as can be seen by Harry's supposed surprise on learning he's a wizard. I'm a wizard! is said with such indifference you'd think he's not surprised at all."
], StringType()).toDF("text")

## Save and load trained pipeline

In [24]:
## if you're running in colab , it is not recommended to run this cell as colab has memory limitations
## run if your running locally

# pipelineModel.save("./sentimentdl_pipeline")
# loadedPipeline = PipelineModel.load("./sentimentdl_pipeline")
# loadedPipeline.transform(load_df)

## Save and load pre-trained SentimentDL model

In [ ]:
# hdfs:/ if you are saving it on distributed file systems in Hadoop
pipelineModel.stages[-1].write().overwrite().save('./tmp_sentimentdl_model')



Let's use our pre-trained SentimentDLModel in a pipeline: 

In [16]:
# In a new pipeline you can load it for prediction
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained() \
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

sentimentdl = SentimentDLModel.load("./tmp_sentimentdl_model") \
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")

pipeline = Pipeline(
    stages = [
        document,
        use,
        sentimentdl
    ])


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


Now let's load it back so we can have prediction all together with everything in that pipeline:

In [21]:
from pyspark.sql.types import StringType

dfTest = spark.createDataFrame([
    "This movie is a delight for those of all ages. I have seen it several times and each time I am enchanted by the characters and magic. The cast is outstanding, the special effects delightful, everything most believable.",
    "This film was to put it simply rubbish. The child actors couldn't act, as can be seen by Harry's supposed surprise on learning he's a wizard. I'm a wizard! is said with such indifference you'd think he's not surprised at all."
], StringType()).toDF("text")

In [22]:
type(dfTest)

pyspark.sql.dataframe.DataFrame

In [23]:
prediction = pipeline.fit(dfTest).transform(dfTest)

In [24]:
prediction.select("class.result").show()

prediction.select("class.metadata").show(truncate=False)

+----------+
|    result|
+----------+
|[positive]|
|[negative]|
+----------+

+-------------------------------------------------------------------+
|metadata                                                           |
+-------------------------------------------------------------------+
|[[sentence -> 0, positive -> 1.0, negative -> 1.5185634E-11]]      |
|[[sentence -> 0, positive -> 1.4495236E-5, negative -> 0.99998546]]|
+-------------------------------------------------------------------+



# Evaluation 

Let's evaluate our SentimentDL model we trained earlier, saved it, and loaded it into a new pipeline by using a test dataset that model has never seen:

In [30]:
testDataset = spark.read \
      .option("header", True) \
      .csv("aclimdb_test.csv")

In [31]:
type(testDataset)

pyspark.sql.dataframe.DataFrame

In [36]:
preds = pipelineModel.transform(testDataset)

In [37]:
preds.select('label','text',"class.result").show(50, truncate=50)

+--------+--------------------------------------------------+----------+
|   label|                                              text|    result|
+--------+--------------------------------------------------+----------+
|negative|The Second Woman is about the story of a myster...|[negative]|
|positive|In my opinion the directing, editing, lighting,...|[positive]|
|positive|I am listening to Istanbul, intent, my eyes clo...|[positive]|
|negative|Before I speak my piece, I would like to make a...|[positive]|
|positive|ManBearPig is a pretty funny episode of South P...|[positive]|
|negative|A buddy and I went to see this movie when it ca...|[negative]|
|negative|It is incredible that there were two films with...|[negative]|
|negative|Dire! Dismal! Awful! Laughable! Disappointing!<...|[negative]|
|positive|HLOTS was an outstanding series, its what NYPD ...|[positive]|
|negative|This is just one of those films which cannot ju...|[negative]|
|negative|This movie had the potential to be a very

In [38]:
preds_df = preds.select('label','text',"class.result").toPandas()

In [33]:
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

The `SentimentDL` has the ability to accept a threshold to set a label on any result that is less than that number. 

For instance, by default the threshold is set on `0.6` and everything below that will be assigned as `neutral`. You can change this label with `setThresholdLabel`.

We need to filter `neutral` results since we don't have any in the original test dataset to compare with.

In [21]:
preds_df = preds_df[preds_df['result'] != 'neutral']


In [42]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report , plot_confusion_matrix

Let's use `classification_report` from `sklearn` to evaluate the final scores. (keep in mind due to limited resources on a free Google Colab we only used 5 Epochs :)

In [46]:
print (classification_report(preds_df['result'], preds_df['label']))

              precision    recall  f1-score   support

    negative       0.84      0.87      0.86     11957
    positive       0.87      0.85      0.86     12749

    accuracy                           0.86     24706
   macro avg       0.86      0.86      0.86     24706
weighted avg       0.86      0.86      0.86     24706



In [40]:
preds_df.tail()

,label,text,result
24995,positive,This show has an amazing plot with good and re...,negative
24996,positive,I don't know what Dick steel was talking about...,negative
24997,positive,Aaliyah blows all the female cast members out ...,positive
24998,negative,the movie is complete disaster. i don't know w...,negative
24999,negative,"First, I am a big fan of Alien and Alien II - ...",negative


In [65]:
def test_sentiment_func():
    test_input=input()
    test_sentence_df = spark.createDataFrame([test_input],StringType()).toDF("text")
    test_pred = pipeline.fit(test_sentence_df).transform(test_sentence_df)
    test_ans_df=test_pred.select("class.result").toPandas()
    print(test_ans_df)

    

In [62]:
# test_sentence_df = spark.createDataFrame([
#     "This movie is a delight for those of all ages. I have seen it several times and each time I am enchanted by the characters and magic. The cast is outstanding, the special effects delightful, everything most believable.",
#     "This film was to put it simply rubbish. The child actors couldn't act, as can be seen by Harry's supposed surprise on learning he's a wizard. I'm a wizard! is said with such indifference you'd think he's not surprised at all."
# ], StringType()).toDF("text")

In [67]:
test_sentiment_func()

today is a great day to work
       result
0  [positive]
